# Chat Models

聊天模型是语言模型的一种变体。虽然聊天模型在底层使用语言模型，但它们使用的界面略有不同。他们没有使用文本输入和文本输出API，而是使用聊天消息作为输入和输出的接口。

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain.globals import set_debug

load_dotenv(find_dotenv())
set_debug(False)



```python
pip install -qU langchain-openai
pip install -qU langchain-anthropic
pip install -qU langchain-google-vertexai
pip install -qU langchain-cohere
pip install -qU langchain-fireworks
pip install -qU langchain-mistralai
pip install -qU langchain-openai
```

聊天模型界面基于消息而不是原始文本。LangChain目前支持的消息类型有`AIMessage`、`HumanMessage`、`SystemMessage`、`FunctionMessage`和`ChatMessage`。`ChatMessage`接受任意角色参数。大多数时候，你只需要处理`HumanMessage`, `AIMessage`和`SystemMessage`

## LCEL
聊天模型实现了` Runnable interface`，这是语言链表达语言(LCEL)的基本构建块。这意味着它们支持`invoke`, `ainvoke`, `stream`, `aststream`, `batch`,` abbatch`, `aststream`日志调用。

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()
messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="What is the purpose of model regularization?"),
]
chat.invoke(messages)
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

chat.batch([messages])
await chat.ainvoke(messages)
async for chunk in chat.astream(messages):
    print(chunk.content, end="", flush=True)
async for chunk in chat.astream_log(messages):
    print(chunk)

## generate
批处理调用，更丰富的输出您可以更进一步，使用`generate`为多组消息生成补全。这将返回带有附加`message`的`LLMResult`。除了返回的消息之外，这将包括关于每代的附加信息(例如完成原因)和关于完整API调用的附加信息(例如使用的令牌总数)。

In [ ]:
batch_messages = [
    [
        SystemMessage(
            content="You are a helpful assistant that translates English to French."
        ),
        HumanMessage(content="I love programming."),
    ],
    [
        SystemMessage(
            content="You are a helpful assistant that translates English to French."
        ),
        HumanMessage(content="I love artificial intelligence."),
    ],
]
result = chat.generate(batch_messages)
result

## 消息类型
ChatModels将消息列表作为输入并返回一条消息。有几种不同类型的消息。所有消息都有一个`role`和一个`content`属性。`role`描述了谁的消息。对于不同的角色，LangChain有不同的消息类。`content`属性描述消息的内容。这可以是一些不同的事情
- 字符串(大多数模型处理这种类型的内容)
- 字典列表(用于多模式输入，其中字典包含有关输入类型和输入位置的信息)

此外，消息还有一个`additional_kwargs`属性。在这里可以传递关于消息的附加信息。这主要用于特定于提供程序而非通用的输入参数。最著名的例子是OpenAI的`function_call `。
- HumanMessage 这表示来自用户的消息。一般只由内容组成
- AIMessage  这表示来自模型的消息。这可能会有`additional_kwargs ` -例如，如果使用OpenAI工具调用`tool_calls `。
- SystemMessage 这表示一个系统消息，它告诉模型如何行为。这通常只包括内容。并不是每个模型都支持这一点。
- FunctionMessage 这表示函数调用的结果。除了`role`和`content`之外，该消息还有一个name参数，该参数表示为产生此结果而调用的函数的名称。
- ToolMessage 这表示工具调用的结果。为了匹配OpenAI的`function`和`tool`消息类型，这与`FunctionMessage`不同。除了`role`和`content`之外，该消息还有一个`tool_call_id `参数，该参数传递了为产生此结果而调用的工具的调用id。


## Streaming
所有`chatmodel`都实现了`Runnable`接口，该接口与所有方法的默认实现一起提供。`Ainvoke`,`batch`, `abbatch`, `stream`, `astream`。这为所有`chatmodel`提供了对流的基本支持。

查看哪些[ integrations ](https://python.langchain.com/docs/integrations/chat/)支持逐个令牌的stream。

In [ ]:
from langchain_community.chat_models import ChatAnthropic
chat = ChatAnthropic(model="claude-2")
for chunk in chat.stream("Write me a song about goldfish on the moon"):
    print(chunk.content, end="", flush=True)

## Tool calling
> 我们交替使用术语工具调用和函数调用。虽然函数调用有时指的是对单个函数的调用，但我们将所有模型视为它们可以在每个消息中返回多个工具或函数调用。

工具调用包括名称、参数字典和可选标识符。参数字典是结构化的`{argument_name: argument_value}.`。

许多LLM提供商，包括人类，Cohere，Google，Mistral，OpenAI等，都支持工具调用功能的变体。这些功能通常允许向LLM的请求包含可用工具及其模式，并供应包括对这些工具的呼叫。例如，给定搜索引擎工具，LLM可能首先发出搜索引擎的调用来处理查询。呼叫LLM的系统可以接收工具调用，执行该工具并将输出返回LLM以告知其响应。Langchain包括一套内置工具，并支持定义您自己的自定义工具的几种方法。工具呼叫对于建造工具的链条和代理非常有用，并且对于从模型中获取结构化输出非常有用。

提供程序采用不同的约定来格式化工具模式和工具调用。例如，Anthropic将工具调用作为较大内容块中的解析结构返回


```json
[
  {
    "text": "<thinking>\nI should use a tool.\n</thinking>",
    "type": "text"
  },
  {
    "id": "id_value",
    "input": {"arg_name": "arg_value"},
    "name": "tool_name",
    "type": "tool_use"
  }
]

```

而OpenAI将工具调用分离为一个单独的参数，参数为JSON字符串

```json
{
  "tool_calls": [
    {
      "id": "id_value",
      "function": {
        "arguments": '{"arg_name": "arg_value"}',
        "name": "tool_name"
      },
      "type": "function"
    }
  ]
}

```

LangChain实现了定义工具、将工具传递给llm以及表示工具调用的标准接口。




### Passing tools to LLMs

支持工具调用特性的聊天模型实现了一个`.bind_tools`方法，该方法接收LangChain工具对象列表，并以期望的格式将它们绑定到聊天模型。聊天模型的后续调用将在其对LLM的调用中包含工具模式。

例如，我们可以在Python函数上使用@tool装饰器来定义自定义工具的模式

In [ ]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

下面，我们使用Pydantic定义模式

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field


# 注意，这里的文档字符串至关重要，因为它们将与类名一起传递给模型。
class Add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class Multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [Add, Multiply]

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

llm_with_tools = llm.bind_tools(tools)

### Tool calls
如果工具调用包含在LLM响应中，它们将作为`.tool_calls`属性中的工具调用对象列表附加到相应的消息或消息块中。`ToolCall`是一个类型化字典，它包括工具名称、参数值字典和(可选的)标识符。没有工具的消息默认调用此属性的空列表。

In [ ]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

`.tool_calls`属性应该包含有效的工具调用。请注意，有时，模型提供者可能会输出格式错误的工具调用(例如，不是有效JSON的参数)。当在这些情况下解析失败时，`InvalidToolCall`的实例将被填充到`.invalid_tool_calls`属性中。`InvalidToolCall`可以有名称、字符串参数、标识符和错误消息。

In [ ]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[Multiply, Add])
res= chain.invoke(query)
res

In [ ]:
type(res)

In [ ]:
res[0].schema_json()

### Streaming
当在流上下文中调用工具时，消息块将通过`.tool_call_chunks`属性在列表中填充工具调用块对象。`ToolCallChunk`包含可选的字符串字段，用于工具name、args和id，还包括可选的整数字段index，可用于将块连接在一起。字段是可选的，因为工具调用的部分可能跨不同的块进行流(例如，包含参数的子字符串的块可能具有工具name和id的空值)。

因为消息块继承自它们的父消息类，所以带有工具调用块的`AIMessageChunk`还将包含`.tool_call`和`.invalid_tool_calls`字段。这些字段是从消息的工具调用块中尽力解析出来的。

请注意，并非所有提供程序当前都支持工具调用的流。

In [ ]:
print("----", query)
async for chunk in llm_with_tools.astream(query):
    print(chunk.tool_call_chunks)

注意，添加消息块将合并它们对应的工具调用块。这就是LangChain的各种工具[输出解析器](https://python.langchain.com/docs/modules/model_io/output_parsers/types/openai_tools/)支持流的原理。

In [ ]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_call_chunks)

In [ ]:
print(type(gathered.tool_call_chunks[0]["args"]))

下面我们将累积工具调用来演示部分解析

In [ ]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_calls)

In [ ]:
print(gathered.tool_calls[0]["args"])

In [ ]:
print(gathered.tool_calls)

### Passing tool outputs to model
如果我们使用模型生成的工具调用来实际调用工具，并希望将工具结果传递回模型，我们可以使用`ToolMessages`来实现。

In [ ]:
query

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
messages

In [ ]:
ai_msg.tool_calls

In [ ]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    # print("-----", tool_output)
    # print("-----", tool_call["id"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
messages

In [ ]:
llm_with_tools.invoke(messages)

### Few-shot prompting
对于更复杂的工具使用，在提示符中添加几个示例非常有用。我们可以通过在提示符中添加带有`toolcall`的`AIMessages`和相应的`ToolMessages`来实现这一点。

例如，即使有一些特殊的指令，我们的模型也会因为操作顺序而出错

In [ ]:
llm_with_tools.invoke(
    "Whats 119 times 8 minus 20. Don't do any math yourself, only use tools for math. Respect order of operations"
).tool_calls

模型不应该尝试添加任何东西，因为从技术上讲，它还不能知道119 * 8的结果。

通过添加一些示例提示，我们可以纠正这种行为:

In [ ]:
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

examples = [
    HumanMessage(
        "What's the product of 317253 and 128472 plus four", 
        name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "Multiply", "args": {"x": 317253, "y": 128472}, "id": "1"}
        ],
    ),
    ToolMessage("16505054784", 
                tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "Add", "args": {"x": 16505054784, "y": 4}, "id": "2"}],
    ),
    ToolMessage("16505054788", 
                tool_call_id="2"),
    AIMessage(
        "The product of 317253 and 128472 plus four is 16505054788",
        name="example_assistant",
    ),
]

system = """You are bad at math but are an expert at using a calculator. 

Use past tool usage as an example of how to correctly use the tools."""
few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        *examples,
        ("human", "{query}"),
    ]
)

chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools
chain.invoke("Whats 119 times 8 minus 20").tool_calls

## 结构化输出
通常，让LLM返回结构化产出是至关重要的。这是因为llm的输出通常用于需要特定参数的下游应用程序。为此，让LLM可靠地返回结构化输出是必要的。

有几种不同的高级策略可以用来做到这一点
- 提示:这是当您要求LLM(非常友好地)以所需格式(JSON, XML)返回输出时。这很好，因为它适用于所有LLM。这并不好，因为不能保证LLM以正确的格式返回输出。
- 函数调用:这时，LLM被微调为不仅能够生成补全，而且能够生成函数调用。LLM可以调用的函数通常作为额外参数传递给模型API。函数名和描述应该被视为提示符的一部分(它们通常被计入令牌计数，并被LLM用来决定要做什么)。
- 工具调用:一种类似于函数调用的技术，但它允许LLM同时调用多个函数。
- JSON模式:LLM保证返回JSON。

不同的模型可能支持这些参数的不同变体，参数略有不同。为了使llm更容易返回结构化输出，我们为LangChain模型添加了一个通用接口:`.with_structured_output`。

通过调用此方法(并传入JSON模式或Pydantic模型)，模型将添加任何模型参数和输出解析器，以返回结构化输出。可能有不止一种方法可以做到这一点(例如，函数调用与JSON模式)-你可以通过传入该方法来配置使用哪个方法。

让我们来看一些实际的例子!我们将使用Pydantic轻松构建响应模式。


In [39]:
from langchain_core.pydantic_v1 import BaseModel, Field


class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")

### OpenAI
OpenAI公开了几种不同的方式来获得结构化的输出。


In [ ]:
from langchain_openai import ChatOpenAI

默认情况下，我们将使用`function_calling`

In [40]:
model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = model.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")

Joke(setup='Why was the cat sitting on the computer?', punchline='To keep an eye on the mouse!')

In [41]:
structured_llm

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7faca82d5f30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7faca82d5db0>, model_name='gpt-3.5-turbo-0125', temperature=0.0, openai_api_key=SecretStr('**********'), openai_api_base='https://flag.smarttrot.com/v1', openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Joke', 'description': '', 'parameters': {'type': 'object', 'properties': {'setup': {'description': 'The setup of the joke', 'type': 'string'}, 'punchline': {'description': 'The punchline to the joke', 'type': 'string'}}, 'required': ['setup', 'punchline']}}}], 'tool_choice': {'type': 'function', 'function': {'name': 'Joke'}}})
| PydanticToolsParser(first_tool_only=True, tools=[<class '__main__.Joke'>])

### JSON Mode
我们还支持JSON模式。注意，我们需要在提示符中指定它应该响应的格式。


In [42]:
structured_llm = model.with_structured_output(Joke, method="json_mode")
structured_llm.invoke(
    "Tell me a joke about cats, respond in JSON with `setup` and `punchline` keys"
)

Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!')

### Fireworks
Fireworks同样支持函数调用和选择模型的JSON模式。

默认情况下，我们将使用`function_calling`

In [45]:
from langchain_fireworks import ChatFireworks
model = ChatFireworks(model="accounts/fireworks/models/firefunction-v1")
structured_llm = model.with_structured_output(Joke)


/Users/dyz/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The method `ChatFireworks.with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [47]:
structured_llm.invoke("Tell me a joke about cats")

Joke(setup="Why don't cats play poker in the jungle?", punchline='Too many cheetahs!')

#### JSON Mode
我们还支持JSON模式。注意，我们需要在提示符中指定它应该响应的格式。

In [48]:
structured_llm = model.with_structured_output(Joke, method="json_mode")
structured_llm.invoke(
    "Tell me a joke about dogs, respond in JSON with `setup` and `punchline` keys"
)

Joke(setup='Why did the dog sit in the shade?', punchline='To avoid getting burned.')

#### Mistral
我们还支持Mistral模型的结构化输出，尽管我们只支持函数调用。

In [49]:
from langchain_mistralai import ChatMistralAI
model = ChatMistralAI(model="mistral-large-latest")
structured_llm = model.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")


LocalProtocolError: Illegal header value b'Bearer '

### Together
因为TogetherAI只是OpenAI的一个替代品，我们可以只使用OpenAI集成


In [51]:
import os

from langchain_openai import ChatOpenAI
model = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
)
structured_llm = model.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")

Joke(setup='Why did the cat sit on the computer?', punchline='To keep an eye on the mouse!')

### Groq
Groq提供了一个与openai兼容的函数调用API。

In [56]:
from langchain_groq import ChatGroq
model = ChatGroq()
structured_llm = model.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")


/Users/dyz/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The method `ChatGroq.with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


Joke(setup="Why don't cats play poker in the jungle?", punchline='Too many cheetahs!')

### JSON Mode
我们还支持JSON模式。注意，我们需要在提示符中指定它应该响应的格式。


In [57]:
structured_llm = model.with_structured_output(Joke, method="json_mode")
structured_llm.invoke(
    "Tell me a joke about cats, respond in JSON with `setup` and `punchline` keys"
)

Joke(setup="Why don't cats play poker in the jungle?", punchline='Too many cheetahs!')

### Anthropic
Anthropic的工具调用API可用于结构化输出。请注意，目前没有办法通过API强制执行工具调用，因此正确提示模型仍然很重要。


In [59]:
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-opus-20240229", temperature=0)
structured_llm = model.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats. Make sure to call the Joke function.")

/Users/dyz/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The method `ChatAnthropic.with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}

### Google Vertex AI
Google的Gemini模型支持函数调用，我们可以通过Vertex AI访问它并使用它来构建输出

In [61]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model="gemini-pro", temperature=0)
structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")

GoogleAuthError: Unable to find your project. Please provide a project ID by:
- Passing a constructor argument
- Using vertexai.init()
- Setting project using 'gcloud config set project my-project'
- Setting a GCP environment variable
- To create a Google Cloud project, please follow guidance at https://developers.google.com/workspace/guides/create-project

## Caching
LangChain为聊天模型提供了一个可选的缓存层。这有两个有用的原因

如果您经常多次请求相同的完成，它可以通过减少对LLM提供程序的API调用数量来节省资金。它可以通过减少对LLM提供程序的API调用数量来加快应用程序的速度。

In [64]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
# <!-- ruff: noqa: F821 -->
from langchain.globals import set_llm_cache

### In Memory Cache

In [65]:
%%time
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())

# The first time, it is not yet in cache, so it should take longer
llm.predict("Tell me a joke")

/Users/dyz/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


CPU times: user 80.1 ms, sys: 25.3 ms, total: 105 ms
Wall time: 5.07 s


'Why did the scarecrow win an award?\nBecause he was outstanding in his field!'

In [66]:
%%time
# The second time it is, so it goes faster
llm.predict("Tell me a joke")

CPU times: user 2.09 ms, sys: 3.36 ms, total: 5.45 ms
Wall time: 5.3 ms


'Why did the scarecrow win an award?\nBecause he was outstanding in his field!'

### SQLite Cache


In [67]:
# We can do the same thing with a SQLite cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [68]:
%%time
# The first time, it is not yet in cache, so it should take longer
llm.predict("Tell me a joke")

CPU times: user 36 ms, sys: 41.9 ms, total: 77.9 ms
Wall time: 3.66 s


"Why couldn't the bicycle stand up by itself?\n\nBecause it was two tired!"

In [69]:
%%time
# The second time it is, so it goes faster
llm.predict("Tell me a joke")

CPU times: user 4.58 ms, sys: 10.4 ms, total: 15 ms
Wall time: 17.6 ms


"Why couldn't the bicycle stand up by itself?\n\nBecause it was two tired!"

## Custom Chat Model
在本指南中，我们将学习如何使用LangChain抽象创建自定义聊天模型。

用标准的`BaseChatModel`接口包装LLM，允许您在现有的LangChain程序中使用LLM，只需进行最小的代码修改

作为奖励，您的LLM将自动成为一个可运行的LangChain，并将受益于一些现成的优化(例如，通过线程池进行批处理)，`astream_events`API等。

### Inputs and outputs
首先，我们需要讨论作为聊天模型输入和输出的消息

#### Messages
聊天模型接受消息作为输入，并返回消息作为输出。
LangChain有一些内置的消息类型

表格
| Message Type | Description |
| ------------ | ----------- |
| SystemMessage | 用于启动AI行为，通常作为输入消息序列的第一个传入。 |
| HumanMessage | 表示来自与聊天模型交互的人员的消息。 |
| AIMessage | 表示来自聊天模型的消息。这可以是文本，也可以是调用工具的请求。 |
| FunctionMessage / ToolMessage | 消息，用于将工具调用的结果传递回模型。 |
| AIMessageChunk / HumanMessageChunk / … | 每种消息类型的块变体。 |

> `ToolMessage`和`FunctionMessage`紧跟OpenAIs的功能和工具角色。这是一个快速发展的领域，随着越来越多的模型添加了函数调用功能，预计该模式还会增加一些功能。

In [ ]:
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    FunctionMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)


### Streaming Variant
所有聊天消息都有一个流变体，其名称中包含Chunk。


In [1]:
from langchain_core.messages import (
    AIMessageChunk,
    FunctionMessageChunk,
    HumanMessageChunk,
    SystemMessageChunk,
    ToolMessageChunk,
)

当从聊天模型流输出时使用这些块，它们都定义了一个附加属性

In [2]:
AIMessageChunk(content="Hello") + AIMessageChunk(content=" World!")

AIMessageChunk(content='Hello World!')

## [获取对数概率](https://python.langchain.com/docs/modules/model_io/chat/logprobs/)
可以将某些聊天模型配置为返回令牌级日志概率。本指南介绍了如何获取许多模型的logprobs。
### OpenAI
安装LangChain x OpenAI包并设置API密钥
为了让OpenAI API返回日志概率，我们需要配置`logprobs=True`参数


In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"],
                 base_url=os.environ["OPENAI_API_BASE"]).bind(logprobs=True)

msg = llm.invoke(("human", "how are you today"))

In [8]:
msg

AIMessage(content="I'm just a computer program, so I don't have feelings or emotions like humans do. I'm here to help answer your questions to the best of my abilities. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 16, 'total_tokens': 57}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': {'content': [{'token': 'I', 'bytes': [73], 'logprob': -0.23781902, 'top_logprobs': []}, {'token': "'m", 'bytes': [39, 109], 'logprob': -0.42825982, 'top_logprobs': []}, {'token': ' just', 'bytes': [32, 106, 117, 115, 116], 'logprob': -0.16595864, 'top_logprobs': []}, {'token': ' a', 'bytes': [32, 97], 'logprob': -0.001797635, 'top_logprobs': []}, {'token': ' computer', 'bytes': [32, 99, 111, 109, 112, 117, 116, 101, 114], 'logprob': -0.05975598, 'top_logprobs': []}, {'token': ' program', 'bytes': [32, 112, 114, 111, 103, 114, 97, 109], 'logprob': -9.019238e-05, 'top_logprobs': []}, {'t

作为Response_Metadata的一部分，每个输出消息都包含登录程序：

In [9]:
msg.response_metadata["logprobs"]["content"][:5]

[{'token': 'I', 'bytes': [73], 'logprob': -0.23781902, 'top_logprobs': []},
 {'token': "'m",
  'bytes': [39, 109],
  'logprob': -0.42825982,
  'top_logprobs': []},
 {'token': ' just',
  'bytes': [32, 106, 117, 115, 116],
  'logprob': -0.16595864,
  'top_logprobs': []},
 {'token': ' a',
  'bytes': [32, 97],
  'logprob': -0.001797635,
  'top_logprobs': []},
 {'token': ' computer',
  'bytes': [32, 99, 111, 109, 112, 117, 116, 101, 114],
  'logprob': -0.05975598,
  'top_logprobs': []}]

In [10]:
ct = 0
full = None
for chunk in llm.stream(("human", "how are you today")):
    if ct < 5:
        full = chunk if full is None else full + chunk
        if "logprobs" in full.response_metadata:
            print(full.response_metadata["logprobs"]["content"])
    else:
        break
    ct += 1

[]
[{'token': 'I', 'bytes': [73], 'logprob': -0.23781902, 'top_logprobs': []}]
[{'token': 'I', 'bytes': [73], 'logprob': -0.23781902, 'top_logprobs': []}, {'token': "'m", 'bytes': [39, 109], 'logprob': -0.42825982, 'top_logprobs': []}]
[{'token': 'I', 'bytes': [73], 'logprob': -0.23781902, 'top_logprobs': []}, {'token': "'m", 'bytes': [39, 109], 'logprob': -0.42825982, 'top_logprobs': []}, {'token': ' just', 'bytes': [32, 106, 117, 115, 116], 'logprob': -0.16595864, 'top_logprobs': []}]
[{'token': 'I', 'bytes': [73], 'logprob': -0.23781902, 'top_logprobs': []}, {'token': "'m", 'bytes': [39, 109], 'logprob': -0.42825982, 'top_logprobs': []}, {'token': ' just', 'bytes': [32, 106, 117, 115, 116], 'logprob': -0.16595864, 'top_logprobs': []}, {'token': ' a', 'bytes': [32, 97], 'logprob': -0.001797635, 'top_logprobs': []}]


### Response metadata
许多模型提供程序在其聊天生成响应中包含一些元数据。这个元数据可以通过`AIMessage.response_metadata: Dict`:属性。根据模型提供程序和模型配置，这可能包含[令牌计数](https://python.langchain.com/docs/modules/model_io/chat/token_usage_tracking/)、[日志事件](https://python.langchain.com/docs/modules/model_io/chat/logprobs/)等信息。

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo")
msg = llm.invoke([("human", "What's the oldest known example of cuneiform")])
msg.response_metadata

{'token_usage': {'completion_tokens': 149,
  'prompt_tokens': 17,
  'total_tokens': 166},
 'model_name': 'gpt-4-turbo',
 'system_fingerprint': 'fp_ea6eb70039',
 'finish_reason': 'stop',
 'logprobs': None}

### Anthropic


In [12]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-sonnet-20240229")
msg = llm.invoke([("human", "What's the oldest known example of cuneiform")])
msg.response_metadata

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}

### Google VertexAI

In [14]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model="gemini-pro")
msg = llm.invoke([("human", "What's the oldest known example of cuneiform")])
msg.response_metadata

GoogleAuthError: Unable to find your project. Please provide a project ID by:
- Passing a constructor argument
- Using vertexai.init()
- Setting project using 'gcloud config set project my-project'
- Setting a GCP environment variable
- To create a Google Cloud project, please follow guidance at https://developers.google.com/workspace/guides/create-project

### Bedrock (Anthropic)


In [ ]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(model_id="anthropic.claude-v2")
msg = llm.invoke([("human", "What's the oldest known example of cuneiform")])
msg.response_metadata

### MistralAI

In [ ]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI()
msg = llm.invoke([("human", "What's the oldest known example of cuneiform")])
msg.response_metadata

### Groq


In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq()
msg = llm.invoke([("human", "What's the oldest known example of cuneiform")])
msg.response_metadata

### TogetherAI


In [ ]:
import os

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
)
msg = llm.invoke([("human", "What's the oldest known example of cuneiform")])
msg.response_metadata

### FireworksAI

In [ ]:
from langchain_fireworks import ChatFireworks

llm = ChatFireworks(model="accounts/fireworks/models/mixtral-8x7b-instruct")
msg = llm.invoke([("human", "What's the oldest known example of cuneiform")])
msg.response_metadata

## [跟踪令牌使用情况](https://python.langchain.com/docs/modules/model_io/chat/token_usage_tracking/)

本笔记本介绍了如何跟踪特定呼叫的令牌使用情况。
### Using AIMessage.response_metadata
许多模型提供程序返回令牌使用信息，作为聊天生成响应的一部分。当可用时，它包含在`AIMessage.response_metadata.`。下面是OpenAI的一个示例

In [ ]:
# !pip install -qU langchain-openai

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo")
msg = llm.invoke([("human", "What's the oldest known example of cuneiform")])
msg.response_metadata

In [ ]:
# !pip install -qU langchain-anthropic

from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-sonnet-20240229")
msg = llm.invoke([("human", "What's the oldest known example of cuneiform")])
msg.response_metadata

### Using callbacks
还有一些特定于api的回调上下文管理器允许您跨多个调用跟踪令牌使用情况。它目前只在OpenAI API和Bedrock Anthropic API中实现。

#### OpenAI
让我们首先看一个非常简单的示例，它跟踪单个Chat模型调用的令牌使用情况。


In [4]:
# !pip install -qU langchain-community wikipedia
from langchain_openai import ChatOpenAI
from langchain_community.callbacks.manager import get_openai_callback
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)
with get_openai_callback() as cb:
    result = llm.invoke("Tell me a joke")
    print(cb)

Tokens Used: 26
	Prompt Tokens: 11
	Completion Tokens: 15
Successful Requests: 1
Total Cost (USD): $0.00056


上下文管理器中的任何东西都会被跟踪。下面是使用它按顺序跟踪多个调用的示例。

In [ ]:
with get_openai_callback() as cb:
    result = llm.invoke("Tell me a joke")
    result2 = llm.invoke("Tell me a joke")
    print(cb.total_tokens)

如果使用了包含多个步骤的链或代理，它将跟踪所有这些步骤。

In [6]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, load_tools
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're a helpful assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
tools = load_tools(["wikipedia"])
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, stream_runnable=False
)

我们必须将`stream_runnable=False`设置为令牌计数工作。默认情况下，`AgentExecutor`将对底层代理进行流处理，这样你就可以在通过AgentExecutor流处理事件时获得最精细的结果。流活动。然而，OpenAI在流模型响应时不返回令牌计数，因此我们需要关闭底层流。

In [ ]:
with get_openai_callback() as cb:
    response = agent_executor.invoke(
        {
            "input": "What's a hummingbird's scientific name and what's the fastest bird species?"
        }
    )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

#### Bedrock Anthropic
`get_bedrock_anthropic_callback`的工作非常相似

In [ ]:
# !pip install langchain-aws
from langchain_aws import ChatBedrock
from langchain_community.callbacks.manager import get_bedrock_anthropic_callback

llm = ChatBedrock(model_id="anthropic.claude-v2")

with get_bedrock_anthropic_callback() as cb:
    result = llm.invoke("Tell me a joke")
    result2 = llm.invoke("Tell me a joke")
    print(cb)